In [1]:
try:
  #mount google drive
  from google.colab import drive
  drive_path="/content/drive"
  drive.mount(drive_path)
  drive_folder=drive_path+"/MyDrive/dtm/"
  using_colab=True
except ModuleNotFoundError:
  # Assume we are not on google colab, 
  drive_folder = "data/"
  using_colab=False
  pass

Mounted at /content/drive


# Params and Init



In [2]:
print("Importing    ")
!pip -q install --upgrade pip
# !pip -q install --upgrade tensorflow keras
!pip -q install plyfile deepdiff talos GitPython numba
!pip -q install --upgrade imgaug
import talos
from deepdiff import DeepDiff
import re
import numpy as np
from progressbar import progressbar, ProgressBar
import sys
import os
import cv2
import math
import json
import importlib
import matplotlib.pyplot as plt
import tensorflow as tf
import git
from tensorflow import keras
import keras.backend as K
import datetime
import imgaug.augmenters as iaa
cuda_paths=!ls /usr/local | grep cuda-
os.environ['CUDA_HOME'] = os.path.join("/usr/local/",cuda_paths[-1])
K.manual_variable_initialization(True)

Importing    
     |████████████████████████████████| 1.6MB 8.5MB/s 
     |████████████████████████████████| 66 kB 2.5 MB/s 
     |████████████████████████████████| 53 kB 2.2 MB/s 
     |████████████████████████████████| 166 kB 51.7 MB/s 
     |████████████████████████████████| 5.4 MB 21.4 MB/s 
     |████████████████████████████████| 9.5 MB 50.7 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 2.6 MB 66.6 MB/s 
     |████████████████████████████████| 26.6 MB 65 kB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 0.1.12 requires imgaug<0.2.7,>=0.2.5, but you have imgaug 0.2.9 which is incompatible.
     |████████████████████████████████| 948 kB 7.7 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of t

In [3]:
print("Importing deep_ga")
if using_colab:
    !git -C /content/deep_ga/ pull || git clone https://github.com/InigoMoreno/deep_ga
    sys.path.append('/content/deep_ga')
    import deep_ga  # nopep8
    deep_ga=importlib.reload(deep_ga)
else:
    MODULE_PATH = "deep_ga/deep_ga/__init__.py"
    MODULE_NAME = "deep_ga"
    git.cmd.Git(MODULE_NAME).pull()
    spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
    deep_ga = importlib.util.module_from_spec(spec)
    sys.modules[spec.name] = deep_ga
    spec.loader.exec_module(deep_ga)


Importing deep_ga
fatal: cannot change to '/content/deep_ga/': No such file or directory
Cloning into 'deep_ga'...
remote: Enumerating objects: 553, done.
remote: Counting objects: 100% (553/553), done.
remote: Compressing objects: 100% (224/224), done.
remote: Total 553 (delta 373), reused 504 (delta 324), pack-reused 0
Receiving objects: 100% (553/553), 457.14 KiB | 1.86 MiB/s, done.
Resolving deltas: 100% (373/373), done.


In [7]:
p={
  "resolution" : .5,                     # resolution of map [meters per pixel]
  "mapLength"  : 40,                     # size of one side of the map [meters]
  "minSlopeThreshold"  : 0.5,            # minimum slope to be counted [proportion]
  "maxNanPercentage"   : 5/100,          # maximum percentage of NaNs in a patch [%]
  "minSlopePercentage" : 0/100,          # minimum percentage of slope in a patch [%]
  "maxSlopePercentage" : 100/100,        # maximum percentage of slope in a patch [%]
  "stdPatchShift"          : 15,         # standard deviation of shift between to patches [m]
  "local_resolution" : .5,
  "local_mapLength"  : 40,
  "local_maxNanPercentage": 65/100,
  "local_global_minCcorr": 15/100,
   "booleanDist": False
   
}
p["mapLengthPixels"]=math.ceil(p["mapLength"]/p["resolution"])
p["local_mapLengthPixels"]=math.ceil(p["local_mapLength"]/p["local_resolution"])
deep_ga.set_scale(p["stdPatchShift"])

p["augment_a"] = iaa.Sequential([
    iaa.Add((-4, 4)),
    iaa.PerspectiveTransform(scale=0.01),
    iaa.BlendAlphaSimplexNoise(iaa.Add(0.2), upscale_method="cubic", size_px_max=4),
])

hyperparams = {
    # "input": ["raw", "rawmask", "sobel", "fixsobel", "PConv", "SymConv", "Conv"],
    "input": ["PConv"],
    # "batch_size": [32, 64, 128, 256],
    "batch_size": [4],
    "mobileNet_alpha": [1.4,1.00,0.75,0.5,0.35],
    # "mobileNet_alpha": [0.35],
    # "mobileNet_pooling": ["avg", "max", None],
    "mobileNet_pooling": ["avg"],
    # "mobileNet_weights": ["imagenet"],
    "mobileNet_weights": [None],
    # "firstLayerSize": [4096, 1280, 1000],"
    "firstLayerSize": [0],
    # "dropout": [0, 0.2, 0.5],
    "dropout": [0.01],
    # "secondLayerSize": [1000, 500, 100],
    "secondLayerSize": [1000,500,100,50,0],
    # "activation": ["relu", None],
    "activation": ["relu"],
    "sharedWeights": [True],
    # "sharedWeights": [True],
    # "loss": ["MSE", "MAE", "DOOMSE", "PCL", "BCE"],
    "loss": ["DOOMSE"],
    # "arepeat": [1,2],
    # "los_scale": [1],
    "learnSize": [10],
    "optimizer": ["SGD"],
    "learning_rate": [0.0001],
    # "learning_rate": [0.003,0.0001,0.00003],
    # "learnEnding": [True, False],
    "learnEnding": [False],
    # "displace": [True, False],
    # "loadFolder": [os.path.join(drive_folder,
    #                             "talos_oxia_mobileNet_alpha__secondLayerSize")],
    # "loadBranch": [None, "1_0__0"]
}
# hyperparams["loadBranch"]=[file for file in os.listdir(hyperparams["loadFolder"][0]) if re.match(".*\.h5",file) and "weights" not in file]

keys= [k for k,v in hyperparams.items() if len(v)>1]
experiment_name="talos_tenerife_"+"__".join(keys)
experiment_path = os.path.relpath(os.path.join(drive_folder,experiment_name))
print(experiment_name)
first_hyperparams={k:v[0] for k,v in hyperparams.items()}

talos_tenerife_mobileNet_alpha__secondLayerSize


# Model preparation

In [27]:
values=[hyperparams[k] for k in keys]
params=first_hyperparams
print(" & "+ " & ".join([str(v) for v in values[1]])+" \\\\\\hline")
for i,v0 in enumerate(values[0]):
  params[keys[0]]=v0
  print(v0,end="")
  for j,v1 in enumerate(values[1]):
    params[keys[1]]=v1
    input_a = keras.Input(shape=(
        p["local_mapLengthPixels"], p["local_mapLengthPixels"]), name="patch_a")
    input_b = keras.Input(
        shape=(p["mapLengthPixels"], p["mapLengthPixels"]), name="patch_b")

    model = deep_ga.get_model(params,input_a,input_b)
    model = deep_ga.compile_model(model, 0, params)
    print(" & ",deep_ga.count(model), end="")
    # print(" & ",v1, end="")
  print(" \\\\")

 & 1000 & 500 & 100 & 50 & 0 \\\hline
1.4 &  6156739 &  5260239

KeyboardInterrupt: ignored

In [ ]:
import tensorflow as tf

# Convert Keras model to a tflite model
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.OPTIMIZE_FOR_SIZE]
# converter.representative_dataset = representative_data_gen
converter.target_spec.supported_ops = [
  tf.lite.OpsSet.SELECT_TF_OPS # enable TensorFlow ops.
]
tflite_model = converter.convert()

filename=os.path.join(drive_folder, "a.tflite")
open(filename, 'wb').write(tflite_model);
print(f"{os.path.getsize(filename)/pow(2,20):.2f}")
os.remove(filename)

INFO:tensorflow:Assets written to: /tmp/tmp9lv7d5do/assets


INFO:tensorflow:Assets written to: /tmp/tmp9lv7d5do/assets


20.15



Copy this on the console to prevent termination:
```
function ClickConnect() {
console.log("Working"); 
document
  .querySelector('#top-toolbar > colab-connect-button')
  .shadowRoot.querySelector('#connect')
  .click() 
}
setInterval(ClickConnect, 60000)
```

Interesting resources: 
 - Hyperparameter optimization with talos
 - Tinyfying Neural Network with uTensor or TinyML

